In [1]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.models import autoregressive_integrated_moving_average, linear_regression, support_vector_regression, random_forest, gradient_boosting
from AnalyzeTools.prepare import data_split, model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 123


In [2]:
params_path = './Models/single'
train_size = 0.8
product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])

all_experiments

[['pork', '(중)경략가격집계 - 소,돼지', '돼지 온도체', 'MAX_COST_AMT'],
 ['pork', '(중)경략가격집계 - 소,돼지', '돼지 온도체', 'MIN_COST_AMT'],
 ['pork', '(중)축산유통정보 - 소비자가격', 4304, 'DLPC'],
 ['pork', '(중)축산유통정보 - 소비자가격', 4402, 'DLPC'],
 ['pork', '축평원_돼지 삼겹살 소매가격', '삼겹살', '평균'],
 ['pork', '축평원_돼지 삼겹살 소매가격', '삼겹살', '최고'],
 ['pork', '축평원_돼지 삼겹살 소매가격', '삼겹살', '최저'],
 ['pork', '축평원_돼지수입 삼겹살 소매가격', '수입_돼지고기', '평균'],
 ['pork', '축평원_돼지수입 삼겹살 소매가격', '수입_돼지고기', '최고'],
 ['pork', '축평원_돼지수입 삼겹살 소매가격', '수입_돼지고기', '최저'],
 ['beef', '경략가격집계 - 소,돼지', '소', 'MAX_COST_AMT'],
 ['beef', '경략가격집계 - 소,돼지', '소', 'MIN_COST_AMT'],
 ['beef', '경략가격집계 - 소,돼지', '부분육(쇠고기)', 'MAX_COST_AMT'],
 ['beef', '경략가격집계 - 소,돼지', '부분육(쇠고기)', 'MIN_COST_AMT'],
 ['beef', '축산유통정보 - 소비자가격', 4301, 'DLPC'],
 ['beef', '축산유통정보 - 소비자가격', 4401, 'DLPC'],
 ['beef', '축평원_소 수입 소매가격', '미국산_갈비', '평균'],
 ['beef', '축평원_소 수입 소매가격', '미국산_갈비', '최고'],
 ['beef', '축평원_소 수입 소매가격', '미국산_갈비', '최저'],
 ['beef', '축평원_소 수입 소매가격', '호주산_갈비', '평균'],
 ['beef', '축평원_소 수입 소매가격', '호주산_갈비', '최고'],
 [

In [3]:
n = 0
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

Product: pork
Raw file name: (중)경략가격집계 - 소,돼지
Product_type: 돼지 온도체
target: MAX_COST_AMT


In [4]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

if len(df) == 0:
    raise ValueError("No data!")

df, input_features = preprocessData(df, 'date', target)
predictions_x_axis = df['date'][floor(len(df) * train_size):].values


-->Feature scores:
                 Features        Scores
6  DEFECT_MAX_COST_AMT  6.811574e+10
1         MIN_COST_AMT  1.222592e+04
5  DEFECT_MIN_COST_AMT  1.222443e+04
2         SUM_COST_AMT  4.538033e+02
7  DEFECT_SUM_COST_AMT  4.537454e+02
8    DEFECT_SUM_WEIGHT  2.279391e+01
3           SUM_WEIGHT  2.277102e+01
4           DEFECT_CNT  1.084715e+01
0                  CNT  1.083094e+01

-->TOP K features:
   ['MIN_COST_AMT', 'SUM_COST_AMT', 'DEFECT_MIN_COST_AMT', 'DEFECT_MAX_COST_AMT']

-->Final features:
  ['MIN_COST_AMT', 'SUM_COST_AMT', 'DEFECT_MAX_COST_AMT']


In [5]:
# prepare dataset for statistics and Macnhine models
ml_split_params = {'Model': 'ML', 'Future': 1}
X_train, X_test, y_train, y_test, input_scaler, output_scaler = data_split(df, input_features, target, train_size=0.8, scaling=True, **ml_split_params)

''' Input data into models and Evaluate model results '''
ml_searchCV_params = {
    'base_dir': params_path,
    'product': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'save': True
}
stdout = True
vis = True

X_train: (2037, 4) y_train: (2037,) X_test: (510, 4) y_test: (510,)


# Statistics

In [6]:
# print("\nARIMA")
# arima_predictions = autoregressive_integrated_moving_average(y_train, y_test)
# model_eval(y_test, arima_predictions, stdout=stdout, vis=vis)

In [7]:
print("\nLinear Regression")
lr, _ = linear_regression(X_train, y_train)
lr_predictions = lr.predict(X_test)
model_eval(y_test, lr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Linear Regression
MAPE: 0.04591038814470225 R square: 0.7887017123984879


# Machine learning

In [8]:
print("\nSupport Vector Regression")
svr, _ = support_vector_regression(X_train, y_train, search=True, **ml_searchCV_params)
svr_predictions = svr.predict(X_test)
model_eval(y_test, svr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Support Vector Regression
--> Use the existed best parameters!

Best parameter for SVR is:
  {'C': 10, 'gamma': 0.01, 'epsilon': 0.05, 'kernel': 'rbf'}
MAPE: 0.04610795891242651 R square: 0.7791376046430496


In [9]:
print("\nRandom Forest")
rf, _ = random_forest(X_train, y_train, search=True, **ml_searchCV_params, **{'scaler': output_scaler})
rf_predictions = rf.predict(X_test)
model_eval(y_test, rf_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Random Forest
--> Use the existed best parameters!

Best parameter for Random forest is:
  {'n_estimators': 50, 'max_depth': 7, 'max_features': 1, 'min_samples_leaf': 5, 'min_samples_split': 9}
MAPE: 0.05653457351664959 R square: 0.698031923380585


In [10]:
print("\nGradient Boosting")
gb, _ = gradient_boosting(X_train, y_train, search=True, **ml_searchCV_params)
gb_predictions = gb.predict(X_test)
model_eval(y_test, gb_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Gradient Boosting
--> Use the existed best parameters!

Best parameter for Gradient Boosting is:
  {'n_estimators': 70, 'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 7, 'min_samples_split': 3}
MAPE: 0.056641378295991286 R square: 0.6803308471467395


# Deep learning

In [11]:
data = df.copy()

data['time_idx'] = range(len(data))
data['group'] = product

training_cutoff = floor(len(data) * 0.8)

max_prediction_length = 1
max_encoder_length = 30 # 7, 14, 30, 60, 120
batch_size = 64

group = ['group']
time_varying_known_categoricals = ['month', 'week']
time_varying_unknown_categoricals = []
time_varying_known_reals = ['time_idx']
time_varying_unknown_reals = input_features + [target]

In [18]:
print("\nLSTM")
lstm, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'LSTM'),
    'LSTM'
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
lstm_predictions = lstm.predict(val_dataloader)
model_eval(actuals, lstm_predictions, predictions_x_axis, stdout=True, vis=True)


LSTM
MAPE: 0.04225059971213341 R square: 0.8567336496668998


In [17]:
print("\nGRU")
gru, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'GRU'),
    'GRU'
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
gru_predictions = gru.predict(val_dataloader)

model_eval(actuals, gru_predictions, predictions_x_axis, stdout=True, vis=True)


GRU
MAPE: 0.040665607899427414 R square: 0.8629352526535041


In [16]:
print("\nDeepAR")
deep_ar, val_dataloader = DEEPAR(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'DEEPAR'),
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
deepar_predictions = deep_ar.predict(val_dataloader)

model_eval(actuals, deepar_predictions, predictions_x_axis, stdout=True, vis=True)


DeepAR
MAPE: 0.06209864839911461 R square: 0.7238951345425522


# Transformer

In [15]:
print("\nTFT")
tft, val_dataloader = TFT(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_unknown_categoricals,
    time_varying_known_categoricals,
    time_varying_known_reals,
    time_varying_unknown_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'TFT'),
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
tft_predictions = tft.predict(val_dataloader)

model_eval(actuals, tft_predictions, predictions_x_axis, stdout=True, vis=True)


TFT
MAPE: 0.035624004900455475 R square: 0.8819710035850792
